In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# read csv file
df = pd.read_csv('Load_history.csv')

# drop rows with NaN values
df.dropna(inplace=True)

# replace commas with empty strings and convert to integer
for i in range(1,25):
    var = 'h' + str(i)
    df[var] = df[var].str.replace(',', '').astype(int)

# split into features and target
X = df.iloc[:, :-24]  # features are all columns except the last 24
y = df.iloc[:, -24:]  # target is the last 24 columns

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
X

,zone_id,year,month,day
0,1,2004,1,1
1,1,2004,1,2
2,1,2004,1,3
3,1,2004,1,4
4,1,2004,1,5
...,...,...,...,...
32987,20,2008,6,25
32988,20,2008,6,26
32989,20,2008,6,27
32990,20,2008,6,28


In [3]:
y

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
0,16853,16450,16517,16873,17064,17727,18574,19355,19534,18611,...,13518,13138,14130,16809,18150,18235,17925,16904,16162,14750
1,14155,14038,14019,14489,14920,16072,17800,19089,19577,20047,...,16127,15448,15839,17727,18895,18650,18443,17580,16467,15258
2,14439,14272,14109,14081,14775,15491,16536,18197,19109,18012,...,13507,13414,13826,15825,16996,16394,15406,14278,13315,12424
3,11273,10415,9943,9859,9881,10248,11016,12780,15108,15680,...,14207,13614,14162,16237,17430,17218,16633,15238,13580,11727
4,10750,10321,10107,10065,10419,12101,14847,15259,14045,14009,...,13845,14350,15501,17307,18786,19089,19192,18416,17006,16018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32987,73065,67894,64837,64404,65531,71468,79852,86082,88122,92356,...,102004,107299,112515,116978,127149,122255,116526,114659,102275,88129
32988,78902,73387,69721,68336,69365,76007,84527,91145,95195,99966,...,118403,123263,127387,132327,141619,136113,130257,128428,114889,100105
32989,89064,81762,77893,75500,75549,80140,87649,96263,102700,109845,...,111177,116193,122072,124814,131446,125277,118750,117213,107443,94293
32990,84385,78279,74501,72190,72121,73491,77042,86519,97277,106064,...,125199,119400,123975,130909,119595,111219,106798,104711,97878,87845


In [4]:
from sklearn.ensemble import RandomForestRegressor

# create model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# train model
rf_model.fit(X_train, y_train)
# rf_model.fit(X, y)

# make predictions
y_pred = rf_model.predict(X_test)


In [5]:
from sklearn.metrics import mean_absolute_error

# calculate MAE
mae = mean_absolute_error(y_test, y_pred)

print("Mean Absolute Error:", mae)

Mean Absolute Error: 6972.294033864025


In [6]:
y_test

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
24757,42664,43487,44502,45659,46495,47172,52115,51882,47970,45531,...,32402,33746,36210,40694,42898,42711,41600,39842,38245,35656
25579,26041,25166,24956,25389,26254,27695,30330,34632,36932,36523,...,24857,23104,22442,22573,23607,25823,29413,29883,27833,25228
14413,81165,81165,80325,77322,73521,64617,64827,65037,64953,64974,...,78519,74424,78057,78015,78372,78225,78267,72051,64785,65310
31527,65712,61074,57787,56332,56098,56946,60823,67442,74132,79304,...,88055,89680,92096,93029,90385,84518,79476,79440,77389,68334
4890,137863,130353,128189,129331,131503,140163,160632,173072,174158,175865,...,170502,168552,167566,167567,168116,168208,170492,166459,156028,140543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32700,73166,69441,66693,65774,67112,74163,86554,90508,88455,90637,...,90209,92859,97115,110011,108197,104506,106296,98652,86139,74836
19443,133332,118048,110777,104714,104325,111032,124202,138616,155080,176968,...,257607,269503,279403,283723,275542,255975,239572,227479,199365,169572
1346,18149,16383,14902,13989,13643,13611,14429,15443,17604,19853,...,33595,34839,35512,35414,33391,30862,29349,25873,22631,19602
32849,99203,98115,97392,97786,100122,107788,123236,117438,117487,108625,...,84828,84567,87159,84356,94265,105135,105036,104475,100555,95978


In [7]:
y_pred

array([[ 37923.86,  38064.02,  38601.9 , ...,  44435.81,  43767.15,
         41677.12],
       [ 19144.82,  18019.91,  17878.26, ...,  29106.37,  27141.2 ,
         24719.29],
       [ 74920.02,  75775.56,  76043.94, ...,  77244.93,  78517.53,
         80630.55],
       ...,
       [ 17269.41,  15718.01,  14658.6 , ...,  25945.38,  22773.63,
         19872.93],
       [ 77288.  ,  75731.6 ,  75767.61, ..., 108883.3 , 101187.95,
         94823.12],
       [100107.35,  99441.89,  99879.78, ..., 106946.99, 105520.15,
         99409.14]])

In [8]:
# read new csv file
new_df = pd.read_csv('Benchmark.csv')

# split into features and target
X_new = new_df.iloc[:, 1:-24]  # features are all columns except the last 24
y_new = new_df.iloc[:, -24:]  # target is the last 24 columns

# make predictions
y_pred_new = rf_model.predict(X_new)

In [9]:
from sklearn.metrics import mean_absolute_error

# calculate MAE
mae = mean_absolute_error(y_new, y_pred_new)

print("Mean Absolute Error:", mae)

Mean Absolute Error: 88385.36602072309


for months:

In [10]:
#for month
df['sum'] =0

# replace commas with empty strings and convert to integer
for i in range(1,25):
    var = 'h' + str(i)
    df['sum'] = df['sum'] + df[var]

print(df.head())
df_month = pd.DataFrame(columns= ['zone_id','year','month'])

x = 0
for i in range(2004,2009):
  for z in range(1,21): 
    for j in range(1,13):
      df_month.at[x,'year'] = i
      df_month.at[x,'zone_id'] = z
      df_month.at[x,'month'] = j
      x += 1

for i in range(1,31):
    var = 'd' + str(i)
    df_month[var] = 0


for d in range(1,32):
  for y in range(2004,2009):
    for m in range(1,13):
      for  z in  range(1, 21):      
                # df_subset = df.loc[(df['year'] == y) & (df['month'] == m) & (df['day'] == d) & (df['zone_id'] == z)]
                # if not df_subset.empty and not pd.isna(df_subset['sum']).values[0]:
                #     df_month.loc[(df_month['year'] == y) & (df_month['month'] == m) & (df_month['zone_id'] == z), 'd'+ str(d)] = int(df_subset['sum'])
                # else:
                #     break

                 df_subset = df.loc[(df['year'] == y) & (df['month'] == m) & (df['day'] == d) & (df['zone_id'] == z)]
                 df_subset = df_subset.dropna(subset=['sum'])  # drop empty cells in the 'sum' column
                 if not df_subset.empty:
                    df_month.loc[(df_month['year'] == y) & (df_month['month'] == m) & (df_month['zone_id'] == z), 'd' + str(d)] = int(df_subset['sum'])

print(df_month.head())

df_month.to_csv("month.csv")

   zone_id  year  month  day     h1     h2     h3     h4     h5     h6  ...  \
0        1  2004      1    1  16853  16450  16517  16873  17064  17727  ...   
1        1  2004      1    2  14155  14038  14019  14489  14920  16072  ...   
2        1  2004      1    3  14439  14272  14109  14081  14775  15491  ...   
3        1  2004      1    4  11273  10415   9943   9859   9881  10248  ...   
4        1  2004      1    5  10750  10321  10107  10065  10419  12101  ...   

     h16    h17    h18    h19    h20    h21    h22    h23    h24     sum  
0  13138  14130  16809  18150  18235  17925  16904  16162  14750  400880  
1  15448  15839  17727  18895  18650  18443  17580  16467  15258  408157  
2  13414  13826  15825  16996  16394  15406  14278  13315  12424  366696  
3  13614  14162  16237  17430  17218  16633  15238  13580  11727  325465  
4  14350  15501  17307  18786  19089  19192  18416  17006  16018  347519  

[5 rows x 29 columns]
  zone_id  year month      d1      d2      d3      d